In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [0]:
oct_df = spark.read.option("header", True) \
.option("inferSchema", True) \
    .csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv")

nov_df = spark.read.option("header", True) \
    .option("inferSchema", True) \
    .csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv")


events = oct_df.unionByName(nov_df)



In [0]:
products = events.select(
    "product_id",
    "brand",
    "category_code"
).dropDuplicates()

users = events.select(
    "user_id"
).dropDuplicates()


In [0]:
events.display()
products.display()
users.display()


In [0]:
# perform join (inner left right outer )
events_joined = (events.join(products, on="product_id", how="left").join(users, on="user_id", how="left"))
display(events_joined)

In [0]:
user_window = Window.partitionBy("user_id").orderBy("event_time")

events_with_running_total = events_joined.withColumn("running_events", F.row_number().over(user_window))

In [0]:
events_with_running_total.select("user_id", "event_time", "event_type", "running_events").show(5)

In [0]:
conversion_rate = (
    events.groupBy("category_code")
    .agg(
        F.sum(F.when(F.col("event_type") == "purchase", 1).otherwise(0)).alias("purchase"),
        F.sum(F.when(F.col("event_type") == "view", 1).otherwise(0)).alias("view")
    )
    .withColumn(
        "conversion_rate",
        F.when(F.col("view") > 0, (F.col("purchase") / F.col("view")) * 100).otherwise(0)
    )
)
conversion_rate.show()

In [0]:
#from pyspark.sql import SparkSession

#spark = SparkSession.builder.getOrCreate()

# Events table
events = spark.createDataFrame([
    (1, 101, "click"),
    (2, 102, "view"),
    (3, 103, "purchase"),
    (4, 104, "click")
], ["event_id", "product_id", "event_type"])

# Products table
products = spark.createDataFrame([
    (101, "Mobile"),
    (102, "Laptop"),
    (103, "Tablet")
], ["product_id", "product_name"])

# Users table
users = spark.createDataFrame([
    (1, "Pavan"),
    (2, "Aman"),
    (3, "Neha")
], ["user_id", "user_name"])


In [0]:
events.display()
products.display()

users.display()

In [0]:
### Inner join 
events.join(products, "product_id", "inner").show()


In [0]:
### left join
events.join(products, "product_id", "left").show()


In [0]:
### right  join 
events.join(products, "product_id", "right").show()


In [0]:
### outer join 
events.join(products, on ="product_id", how ="outer").show()


Window Function

In [0]:
# Spark session start / get existing session
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import sum, row_number, rank, dense_rank

spark = SparkSession.builder.getOrCreate()


data = [
    (1, "Rahul", "2024-01-01", 100),
    (2, "Rahul", "2024-01-02", 200),
    (3, "Rahul", "2024-01-03", 150),
    (4, "Amit",  "2024-01-01", 300),
    (5, "Amit",  "2024-01-02", 100),
    (6, "Amit",  "2024-01-03", 200)
]

columns = ["order_id", "name", "order_date", "amount"]

df = spark.createDataFrame(data, columns)


windowSpec = Window.partitionBy("name").orderBy("order_date")


rankSpec = Window.partitionBy("name").orderBy(df.amount.desc())


df_result = df.withColumn("running_total", sum("amount")\
    .over(windowSpec)).withColumn("row_number", row_number()\
        .over(windowSpec)) .withColumn("rank", rank().over(rankSpec))\
     .withColumn("dense_rank", dense_rank().over(rankSpec)) 
# Final output
df_result.display() 

simple window function 

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

data = [
    ("Sales", "Aman", 5000),
    ("Sales", "Pavan", 7000),
    ("IT", "Neha", 6000),
    ("IT", "Ravi", 8000)
]

df = spark.createDataFrame(data, ["dept", "name", "salary"])
df.show()

w_total = Window.partitionBy("dept")

df2 = df.withColumn("dept_total", F.sum("salary").over(w_total))
df2.show()

w_rank = Window.partitionBy("dept").orderBy(F.desc("salary"))

df3 = df2 \
    .withColumn("row_number", F.row_number().over(w_rank)) \
    .withColumn("rank", F.rank().over(w_rank)) \
    .withColumn("dense_rank", F.dense_rank().over(w_rank))

df3.show()


User Defined Function 


In [0]:
data = [
    ("Sales", "Aman", 5000),
    ("Sales", "Pavan", 7000),
    ("IT", "Neha", 6000),
    ("IT", "Ravi", 8000)
]

df = spark.createDataFrame(data, ["dept", "name", "salary"])
df.show()


In [0]:
def salary_category(salary):
    if salary >= 7000:
        return "High"
    elif salary >= 6000:
        return "Medium"
    else:
        return "Low"


In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

salary_udf = udf(salary_category, StringType())


In [0]:
df_result = df.withColumn("salary_level", salary_udf("salary"))
df_result.show()
